<a href="https://colab.research.google.com/github/thejourneyofman/keras_medicine_detection/blob/master/02_fineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

classes = ['medicine00', 'medicine01', 'medicine02', 'medicine03', 'medicine04',
           'medicine05', 'medicine06', 'medicine07', 'medicine08', 'medicine09']
nb_classes = len(classes)

batch_size = 32
img_rows, img_cols = 150, 150
channels = 3

train_data_dir = 'train'
validation_data_dir = 'test'

nb_train_samples = 16720
nb_val_samples = 5000
nb_epoch = 50

result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)


if __name__ == '__main__':

    # initialize the training data augmentation object
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    # initialize the validation/testing data augmentation object
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    # initialize the training generator
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)

    # initialize the validation generator
    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)

    # load the VGG16 network, ensuring the head FC layer sets are left
    # off
    input_tensor = Input(shape=(img_rows, img_cols, 3))
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)
    # vgg16.summary()

    # construct the head of the model that will be placed on top of the
    # the base model
    head_model = Sequential()
    head_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
    head_model.add(Dense(256, activation='relu'))
    head_model.add(Dropout(0.5))
    head_model.add(Dense(nb_classes, activation='softmax'))

    # Load the trained weights of FC layer
    # top_model.load_weights(os.path.join(result_dir, 'bottleneck_fc_model.h5'))

    # place the head FC model on top of the base model (this will become
    # the actual model we will train)
    model = Model(input=vgg16.input, output=head_model(vgg16.output))

    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    for layer in model.layers[:15]:
        layer.trainable = False

    # compile our model (this needs to be done after our setting our
    # layers to being non-trainable
    print("[INFO] compiling model...")
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])


    # Fine-tuning
    print("[INFO] training head...")
    history = model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_val_samples)

    # serialize and save the model to disk
    model.save_weights(os.path.join(result_dir, 'finetuning.h5'))
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(os.path.join(result_dir, 'history_finetuning.txt'), "w") as fp:
        fp.write("epoch\tloss\tacc\tval_loss\tval_acc\n")
        for i in range(nb_epoch):
            fp.write("%d\t%f\t%f\t%f\t%f\n" % (i, loss[i], acc[i], val_loss[i], val_acc[i]))

Using TensorFlow backend.


Found 16720 images belonging to 10 classes.


W0828 12:48:34.737049 140102370645888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0828 12:48:34.776527 140102370645888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0828 12:48:34.783527 140102370645888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0828 12:48:34.826685 140102370645888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Found 5000 images belonging to 10 classes.
58892288/58889256 [==============================] - 5s 0us/step


W0828 12:48:41.415245 140102370645888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0828 12:48:41.416412 140102370645888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0828 12:48:45.319582 140102370645888 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: UserWarning: Update your `Model` call to the Keras 2 API: `Model(i

[INFO] compiling model...
[INFO] training head...
Epoch 1/50
522/522 [==============================] - 9018s 17s/step - loss: 1.8719 - acc: 0.3430 - val_loss: 1.2022 - val_acc: 0.5828
Epoch 2/50
522/522 [==============================] - 903s 2s/step - loss: 1.0891 - acc: 0.6170 - val_loss: 0.7805 - val_acc: 0.7378
Epoch 3/50
522/522 [==============================] - 901s 2s/step - loss: 0.8039 - acc: 0.7153 - val_loss: 0.5910 - val_acc: 0.8111
Epoch 4/50
522/522 [==============================] - 901s 2s/step - loss: 0.6374 - acc: 0.7797 - val_loss: 0.4594 - val_acc: 0.8539
Epoch 5/50
522/522 [==============================] - 899s 2s/step - loss: 0.5179 - acc: 0.8233 - val_loss: 0.3765 - val_acc: 0.8805
Epoch 6/50
522/522 [==============================] - 898s 2s/step - loss: 0.4406 - acc: 0.8467 - val_loss: 0.3150 - val_acc: 0.8929
Epoch 7/50
522/522 [==============================] - 898s 2s/step - loss: 0.3867 - acc: 0.8684 - val_loss: 0.2778 - val_acc: 0.9162
Epoch 8/50
522/52